# CSS Inbound Calls Forecasting

## Importing libraries

In [1]:
import pandas as pd
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
import matplotlib
import seaborn as sns

matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'


# First Iteration (SARIMAX1) - using data from Jan 2019 to Jul 2023

## Data Preparation

In [ ]:
#read in the csv file with raw call data and set datetime column to a date object
df = pd.read_csv('',parse_dates = [''])

#rename columns
df.rename(columns = {''},inplace=True)
df

In [ ]:
#plot call volumes
plt.figure(figsize=[20,6])
sns.lineplot(data = df, x="Month", y="")
plt.xticks(rotation=45)
plt.show()

In [5]:
df.index = pd.to_datetime(df[''])
df = df[""]

### Decomposing the data

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18,8

decomposition = sm.tsa.seasonal_decompose (df, model='additive')
fig = decomposition.plot()
plt.show()

## Model Training

### Parameter selection

In [ ]:
p = d = q = range (0,2)

pdq = list(itertools.product(p,d,q))
seasonal_pdq = [(x[0],x[1],x[2],12) for x in list(itertools.product(p,d,q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1],seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1],seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2],seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2],seasonal_pdq[4]))

In [ ]:
for param in pdq:
  for param_seasonal in seasonal_pdq:
    try:
      mod = sm.tsa.statespace.SARIMAX(df, 
                                      order=param,
                                      seasonal_order = param_seasonal,
                                      enforce_stationarity=False,
                                      enforce_invertibility= False
                                    )
      
      results = mod.fit()

      print('ARIMA {} x {} 12 - AIC: {}'.format(param,param_seasonal,results.aic))
    except:
      continue

### Fitting model to data

In [ ]:
mod = sm.tsa.statespace.SARIMAX(df,
                                order=(1,1,1),
                                seasonal_order=(1,1,1,12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

### Generating predictions with test data

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2023-02-01'), dynamic = False)
pred_ci = pred.conf_int()

ax = df['2019':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label ='Forecasted', alpha=.7, figsize=(14,7))

ax.fill_between(pred_ci.index, pred_ci.iloc[:,0], pred_ci.iloc[:,1],color='k',alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Inbound Volumes')
plt.legend()

plt.show()

### Calculating the Mean Absolute Percentage Error

In [ ]:
y_forecasted = pred.predicted_mean
y_actual = df['2023-02-01':]


def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

print(f"The Mean Absolute Percentage Error is {round(mape(y_actual,y_forecasted),1)}")

### Calculating the Root Mean Squared Error

In [ ]:
def rmse(predictions,actuals):
    return np.sqrt(((predictions - actuals)**2).mean())

print(f'The Root Mean Squared Error is {round(rmse(y_forecasted,y_actual))}')

In [ ]:
y_forecasted

### Forecasting for the next 12 months

In [ ]:
pred_uc = results.get_forecast(steps=17)
pred_ci = pred_uc.conf_int()

ax = df.plot(label="observed", figsize=(14,7))
pred_uc.predicted_mean.plot(ax=ax, label = 'Forecast')
ax.fill_between(pred_ci.index, pred_ci.iloc[:,0], pred_ci.iloc[:,1], color='k',alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Call Volume')

plt.legend()
plt.show()

### Convert predictions to dataframe

In [ ]:
predicted = pd.DataFrame(pred_uc.predicted_mean)
predicted = predicted.rename(columns={"predicted_mean":"Predicted Volumes"})
predicted

## Save the model

In [15]:
import pickle
results.save('SARIMAX1.pickle')

## Load saved model

In [16]:
from statsmodels.iolib.smpickle import load_pickle
new_results = load_pickle('SARIMAX1.pickle')